<a href="https://colab.research.google.com/github/Vamsi-Malineni/Research-work/blob/master/pinn_trial_tf2_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp 
import os
import sys
import scipy.io
import time 


In [ ]:
class pinn:
  def __init__(self,layers,optimizer,ub,lb):
    
    self.model=tf.keras.Sequential()
    
    self.model.add(tf.keras.layers.InputLayer(input_shape=(layers[0],)))
    
    self.model.add(tf.keras.layers.Lambda(lambda X: 2.0*(X-lb)/(ub-lb)-1))
    # Here X should be tf.concat([x_train,y_train,t_train],1) such that scaling 
    # happens properly

    for width in layers[1:]:
      self.model.add(tf.keras.layers.Dense(
          width,activation=tf.nn.tanh,
          kernel_initializer='glorot_normal'
      ))

  # finding the sizes of weights and biases for each layer in the neural network  
    self.sizes_w=[]
    self.sizes_b=[]

    for i,width in enumerate(layers):
      if i!=1:
        self.sizes_w.append(int(width*layers[1]))
        self.sizes_b.append(int(width if i!=0 else layers[1]))
    
    self.dtype=tf.float32

  # Defining two variables that have to be learnt (lambda1,lambda2)
    self.lambda1=tf.Variable([0.0],dtype=self.dtype)
    self.lambda2=tf.Variable([0.0],dtype=self.dtype)

    self.optimizer=optimizer

  def get_parameters(self,numpy=False):
    l1=self.lambda1
    l2=self.lambda2
    if numpy :
      return l1.numpy()[0],l2.numpy()[0]
    return l1,l2

  def f_g_model(self,xtrain,ytrain,ttrain):
    l1,l2=self.get_parameters()
    # Inputs to be watched by gradient tape
    x_f=tf.convert_to_tensor(xtrain,dtype=self.dtype)
    y_f=tf.convert_to_tensor(ytrain,dtype=self.dtype)
    t_f=tf.convert_to_tensor(ttrain,dtype=self.dtype)
    
    with tf.GradientTape(persistent=True) as g:
      g.watch(x_f)
      g.watch(y_f)
      g.watch(t_f)
      # Stacking the input variables and sending them to model
      xf=tf.stack([x_f[:,0],y_f[:,0],t_f[:,0]],axis=1)

      psi_and_p=self.model(xf)

      psi=psi_and_p[:,0:1]
      p=psi_and_p[:,1:2]

      u= g.gradient(psi,y_f)
      v=-g.gradient(psi,x_f)

      ut=g.gradient(u,t_f)
      ux=g.gradient(u,x_f)
      uy=g.gradient(u,y_f)
      
      vt=g.gradient(v,t_f)
      vx=g.gradient(v,x_f)
      vy=g.gradient(v,y_f)

      px=g.gradient(p,x_f)
      py=g.gradient(p,y_f)
    
    uxx=g.gradient(ux,x_f)
    uyy=g.gradient(uy,y_f)
    vxx=g.gradient(vx,x_f)
    vyy=g.gradient(vy,y_f)

    del g
    f = ut+l1*(u*ux+v*uy)+px-l2*(uxx+uyy) 
    g = vt+l1*(u*vx+v*vy)+py-l2*(vxx+vyy)
    
    return u,v,p,f,g
    
    def loss(self,x,y,t,u,v):
      u_pred,v_pred,p_pred,f_pred,g_pred=self.f_g_model(x,y,t)
      
      loss_eq=tf.reduce_mean(tf.square(u-u_pred))+ \
              tf.reduce_mean(tf.square(v-v_pred))+ \
              tf.reduce_mean(tf.square(f_pred))+ \
              tf.reduce_mean(tf.square(g_pred))
      return loss_eq
    
    def training_variables(self):
      variables=self.model.trainable_variables
      variables.extend([self.lambda1,self.lambda2])
      return variables

    def grad(self,x,y,t,u,v):
      with tf.GradientTape() as g:
        loss_value=self.loss(x,y,t,u,v)
      return loss_value,g.gradient(loss_value,self.training_variables())
    
    def get_weights(self):
      weights_and_biases=[]
      
      for layer in self.model.layers[1:]:
        wandb=layer.get_weights()
        w=wandb[0].flatten()
        b=wandb[1]
        weights_and_biases.extend(w)
        weights_and_biases.extend(b)
      weights_and_biases(self.lambda1.numpy())
      weights_and_biases(self.lambda2.numpy())

      return tf.convert_to_tensor(weights_and_biases,dtype=self.dtype)

    def set_weights(self,w):
      


In [9]:
def prepdata(N_train):
    data = scipy.io.loadmat('/content/drive/MyDrive/cylinder_nektar_wake.mat')
           
    U_star = data['U_star'] # N x 2 x T
    t_star = data['t'] # T x 1
    X_star = data['X_star'] # N x 2
    
    N = X_star.shape[0]
    T = t_star.shape[0]
    
    # Rearrange Data 
    XX = np.tile(X_star[:,0:1], (1,T)) # N x T
    YY = np.tile(X_star[:,1:2], (1,T)) # N x T
    TT = np.tile(t_star, (1,N)).T # N x T
    
    UU = U_star[:,0,:] # N x T
    VV = U_star[:,1,:] # N x T
    
    x = XX.flatten()[:,None] # NT x 1
    y = YY.flatten()[:,None] # NT x 1
    t = TT.flatten()[:,None] # NT x 1
    
    u = UU.flatten()[:,None] # NT x 1
    v = VV.flatten()[:,None] # NT x 1
    
    # Training Data    
    idx = np.random.choice(N*T, N_train, replace=False)
    x_train = x[idx,:]
    y_train = y[idx,:]
    t_train = t[idx,:]
    u_train = u[idx,:]
    v_train = v[idx,:]
    
    lb=tf.math.reduce_min(tf.concat([x_train,y_train,t_train],1),keepdims=True,axis=0) 
    ub=tf.math.reduce_max(tf.concat([x_train,y_train,t_train],1),keepdims=True,axis=0)
        

    return x_train,y_train,t_train,u_train,v_train,lb,ub


In [29]:
print(tf.stack([x[:,0],y[:,0],t[:,0]],axis=1))

tf.Tensor(
[[ 1.42424242 -1.18367347  6.5       ]
 [ 6.37373737  0.85714286 10.5       ]
 [ 1.56565657  1.75510204  2.4       ]
 ...
 [ 6.44444444  1.51020408 17.1       ]
 [ 2.2020202   0.69387755  5.4       ]
 [ 2.13131313  0.93877551 15.2       ]], shape=(5000, 3), dtype=float64)


In [28]:
print(tf.concat([x,y,t],1)[0])

tf.Tensor([ 1.42424242 -1.18367347  6.5       ], shape=(3,), dtype=float64)
